In [45]:
from feature.make_dataset import make_data
from feature.after_dataset import accumulate
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


In [46]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [47]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [48]:
train_1 = pd.read_csv('./data/preprocessing/train_quantile0.25.csv')
test_1 = pd.read_csv('./data/preprocessing/test_quantile0.25.csv')
train_3 = pd.read_csv('./data/preprocessing/train_quantile0.75.csv')
test_3 = pd.read_csv('./data/preprocessing/test_quantile0.75.csv')
train_std = pd.read_csv('./data/preprocessing/train_std.csv')
test_std = pd.read_csv('./data/preprocessing/test_std.csv')
# train_mid = pd.read_csv('./data/preprocessing/train_median.csv')
# test_mid = pd.read_csv('./data/preprocessing/test_median.csv')
# train_sem = pd.read_csv('./data/preprocessing/train_sem.csv')
# test_sem = pd.read_csv('./data/preprocessing/test_sem.csv')

In [49]:
train_3.columns = [col+' 3' for col in train_3.columns]
test_3.columns = [col+' 3' for col in test_3.columns]
train_1.columns = [col+' 1' for col in train_1.columns]
test_1.columns = [col+' 1' for col in test_1.columns]
train_std.columns = [col+' std' for col in train_std.columns]
test_std.columns = [col+' std' for col in test_std.columns]
# train_mid.columns = [col+' mid' for col in train_mid.columns]
# test_mid.columns = [col+' mid' for col in test_mid.columns]
# train_sem.columns = [col+' sem' for col in train_sem.columns]
# test_sem.columns = [col+' sem' for col in test_sem.columns]

In [50]:
train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()
# train_mid = train_mid.reset_index()
# test_mid = test_mid.reset_index()
# train_sem = train_sem.reset_index()
# test_sem = test_sem.reset_index()


In [51]:
train = pd.concat([train, train_1, train_3, train_std], axis=1)
test = pd.concat([test, test_1, test_3, test_std], axis=1)

In [52]:
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

In [53]:
train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)

In [54]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적총광량')

In [55]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [56]:
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU", random_seed=113)
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037333
0:	learn: 40.7184677	total: 9.4ms	remaining: 9.39s
100:	learn: 7.8527340	total: 986ms	remaining: 8.78s
200:	learn: 5.6067142	total: 2s	remaining: 7.96s
300:	learn: 4.9664862	total: 2.99s	remaining: 6.94s
400:	learn: 4.6977367	total: 3.96s	remaining: 5.92s
500:	learn: 4.3434528	total: 4.87s	remaining: 4.85s
600:	learn: 4.1191460	total: 5.86s	remaining: 3.89s
700:	learn: 3.8981229	total: 6.85s	remaining: 2.92s
800:	learn: 3.5081550	total: 7.79s	remaining: 1.94s
900:	learn: 3.2652689	total: 8.72s	remaining: 958ms
999:	learn: 3.1221828	total: 9.69s	remaining: 0us
validation rmse: 9.12521808631475
Learning rate set to 0.039126
0:	learn: 40.3252323	total: 11.1ms	remaining: 11.1s
100:	learn: 6.9754696	total: 1.01s	remaining: 9.01s
200:	learn: 5.0427961	total: 1.99s	remaining: 7.9s
300:	learn: 4.5843875	total: 2.98s	remaining: 6.91s
400:	learn: 4.2436538	total: 3.98s	remaining: 5.95s
500:	learn: 4.0011902	total: 4.99s	remaining: 4.97s
600:	learn: 3.7480191	total: 

In [57]:
df_imp = pd.DataFrame({'imp':cat.feature_importances_}, index = cat.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
"('ec관측치', 2.0)",0.000040
"('시간당백색광량', 1.0)",0.000047
"('시간당적색광량', 1.0)",0.000179
"('시간당백색광량', 4.0)",0.000292
"('시간당적색광량', 3.0)",0.000315
...,...
시간당총광량 std,5.102149
월간누적적색광량,10.297923
월간누적백색광량,10.431380
월간누적분무량,20.194500
